In [2]:
# Author: Robert Guthrie
# https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

import utils

from sklearn.utils import shuffle

In [5]:
import nltk
from sklearn.metrics import classification_report, r2_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn_crfsuite import metrics
import torch
import torch.nn as nn

#from torch_model_base import TorchModelBase
#from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from torch_rnn_classifier import TorchRNNDataset, TorchRNNClassifier, TorchRNNModel
import utils
import json

from collections import Counter
from torch import manual_seed

Run with our data

In [6]:
### RUN 
### NOTE: Make sure to copy most up-to-date annotations2.jsonl file to /Vasco/
with open('annotations2.jsonl') as jsonl_file:
    # note: after running data-preprocessing.ipynb this file already has token-level labels
    lines = jsonl_file.readlines()
annot = [json.loads(line) for line in lines]
#print(annot[0])

In [7]:
### RUN
# now convert annotation tokens into list (sentences) of lists (tokens) format for sklearn_crfsuite.CRF
train_sents=[] 
for j in range(0,len(annot)):
    a = annot[j]['tokens']
    # Only add sample if there are annotations
    if annot[j]['spans']!=[]:
        train_sentence = []
        for i in range(0,len(a)):
            if 'label' in a[i]: # only add element if this sample sentence has been labelled 
                token_element = (a[i]['text'],a[i]['label'])
                train_sentence.append(token_element)
        train_sents.append(train_sentence)

In [8]:
# put into format expected above
START_TAG = "<START>"
STOP_TAG = "<STOP>"
all_data = [([a1 for a1,a2 in el],[a2 for a1,a2 in el]) for el in train_sents]
tag_to_ix = {'ORT': 0, 'STRASSE': 1, 'FLAECHE': 2, 'IMMO_TYP': 3, 'QMPREIS': 4, 'TERRASSENGROESSE': 5, 'KAEUFER': 6, 'VERKAEUFER': 7, 'GESAMTPREIS': 8, 'DATUM_VERTRAG': 9, 'DATUM_VERBUECHERUNG': 10, 'O': 11, START_TAG: 12, STOP_TAG: 13}

In [9]:
train_ratio = 0.75
train_test_split = round(0.75*len(all_data) - 0.5) # -0.5 => floor
idx = [i for i in range(0,len(all_data))]
idx_shuffle = shuffle(idx,random_state=0)
#X_shuffle, y_shuffle = [X[auxIdx] for auxIdx in idx_shuffle], [y[auxIdx] for auxIdx in idx_shuffle]
all_data_shuffle = [all_data[auxIdx] for auxIdx in idx_shuffle]
#X_train, X_test, y_train, y_test = X_shuffle[:train_test_split], X_shuffle[train_test_split:], y_shuffle[:train_test_split], y_shuffle[train_test_split:]
train_data, test_data = all_data_shuffle[:train_test_split], all_data_shuffle[train_test_split:]

In [10]:
def prepare_sequence(seq, to_ix): # converts one list of tokens into one tensor of token ids
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    # Inp: torch array (1,nTags) of next tag scores (accumul transmission + emission); Out: single number tensor
    # see crfCalcs.xlsx
   # max_score = vec[0, argmax(vec)]
    max_score = vec[0, torch.argmax(vec,axis=1)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

def prepare_sequences(seqs, to_ix):
    idxs = [prepare_sequence(seq, to_ix) for seq in seqs]
    return torch.tensor(idxs, dtype=torch.long)
    #return idxs

In [11]:
class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix 
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)
       
        self.transitions = self.init_transitions()

        self.hidden = self.init_hidden()

    def init_transitions(self): # Matrix of transition parameters. Entry i,j is the score of transitioning *to* i *from* j.
        # KEEP
        transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size)) # initialize w/ random numbers (5,5) array
        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        transitions.data[tag_to_ix[START_TAG], :] = -10000
        transitions.data[:, tag_to_ix[STOP_TAG]] = -10000   
       # print(transitions)
        return transitions

    def init_hidden(self):
        # KEEP
        return (torch.randn(2, 1, self.hidden_dim // 2), # here are dividing hidden dim by 2; since bidir end up w/ 2x (=4 tot hid layer)
                torch.randn(2, 1, self.hidden_dim // 2))

    def _forward_alg(self, feats):
        # KEEP
        # Inp: feats is tensor (noTokens,noDiffLabelClasses = tagset_size) = (11,5); Out: tensor 1d
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.) # Inp: self.tagset_size=5 ; Out: tensor (1,5)=-10000
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0. # updates START_TAG Id of init_alphas[0] to be = 0

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats: # iterate through each diff token (feat)
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size): # iterate through each tag
                # broadcast the emission score: it is the same regardless of the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size) # feat[next_tag] is tensor sngl no. w/ gradFn="select_bckwd"; 
                # emit score is (1,tagset_size) w/ gradFn="ExpandBackward"
                # note: in this formula view(1,-1) squishes sngl no. into (1,1) array and expand(1,5) expands it into (1,5) array
                ###########################################################################################
                # the ith entry of trans_score is the score of transitioning to next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1) # self.transitions is (5,5) torch array; trans_score is (1,5)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score # next_tag_var is (1,5)
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        # Input: sentence is list (noTokens); Output lstm_feats is tensor (noTokens,noDiffLabelClasses = len(tag_to_ix))
        self.hidden = self.init_hidden() # self.hidden becomes (a,b) where both are tensors (2,1,2) hidden_dim=4 // 2
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1) # embeds is tensor (noTokens,1,embedding_dim) = (11,1,6)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden) # self.hidden becomes (a,b) where both are tensors (2,1,2) hidden_dim=4 // 2
        # lstm_out is (11,1,4) - last dim is = hidd_dim*2 as bidirect LSTM (NOTE: hidd_dim = self.hidden_dim / 2)
        # Note also: by default there is NO BATCHING so lstm_out is (seq, batch, feature) instead of (batch, seq, feature)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim) # "squishes" middle redundant dimension: (11,4)
        lstm_feats = self.hidden2tag(lstm_out) # FCL that maps from (11,4) to (11,5) where 5 is the tagset space
       # print("lstm_feats")
       # print(lstm_feats.shape)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # KEEP
        # Gives the score of a provided tag sequence - see crfCalcs.xlsx
        score = torch.zeros(1)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long), tags]) # add START_TAG to sentence tags
        for i, feat in enumerate(feats): # iterating thru' all tokens of feats; columns are 1 ... nTags
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats): # feats is (11,5) i.e. LSTM produced prob of each token being associated w/ each of the 5 tags
        # KEEP
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                # best_tag_id = argmax(next_tag_var)
                best_tag_id = torch.argmax(next_tag_var,axis=1)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        # best_tag_id = argmax(terminal_var)
        best_tag_id = torch.argmax(terminal_var,axis=1)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        # KEEP
        # Inputs: sent_in is list (noTokens); targets is torchArray (noTokens); Ouput: loss is single number (1d)
        feats = self._get_lstm_features(sentence) # feats is (noTokens,noDiffLabelClasses)
      #  print(feats)
        forward_score = self._forward_alg(feats) # fwd score from model (accum. transmissions + emissions (i.e. feats))
        gold_score = self._score_sentence(feats, tags) # gold score from self.transitions, feats (emissions), seq of tags in example sentence
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # KEEP
        # Inp: sentence (1 example) as list of tokenIds; Out: score (1d), tag_seq (noTokens) with forecasted tagId for each token
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence) # (noTokens,noDiffLabelClasses)
       # print(lstm_feats)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
      #  print(tag_seq)
        return score, tag_seq

In [21]:
EMBEDDING_DIM = 50
HIDDEN_DIM = 50
#tol = 1e-05
#n_iter_no_change = 10
#validation_fraction=0.1

# Make up some training data
#train_data = [(
#    "the wall street journal reported today that apple corporation made money".split(),
#    "B I I I O O O B I O O".split()
#), (
#    "georgia tech is a university in georgia".split(),
#    "B I O O O O B".split()
#)]

## 5 lines below create dict of (wordToken,wordId)
word_to_ix = {}
for sentence, tags in all_data:
    for word in sentence:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix) # i.e. each successive word added gets a successive index

#tag_to_ix = {"B": 0, "I": 1, "O": 2, START_TAG: 3, STOP_TAG: 4}

vocab = list(set([item for auxList,_ in train_data for item in auxList])) + ["$UNK"]

model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM)
#model = TorchLSTM_CRFSequenceLabeler(
#    len(word_to_ix), 
#    tag_to_ix, 
#    EMBEDDING_DIM, 
#    HIDDEN_DIM,
#    vocab,
#    early_stopping=True,
#    eta=0.001)

optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)
#print(model.parameters().__next__())

# Check predictions before training
#with torch.no_grad():
  # precheck_sent = prepare_sequence(train_data[0][0], word_to_ix)
  # precheck_tags = torch.tensor([tag_to_ix[t] for t in train_data[0][1]], dtype=torch.long)
    # print(model(precheck_sent))

# Make sure prepare_sequence from earlier in the LSTM section is loaded
for epoch in range(1):        #300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in train_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix) # sentence_in is single example (i.e. sentence) with tensor of tokens converted to ids
        targets = torch.tensor([tag_to_ix[t] for t in tags], dtype=torch.long) # convert labels classes to ids

        # Step 3. Run our forward pass.
        #print("sent_in length"+str(len(sentence_in)))
        #print(targets.shape)
        #print(sentence_in)
        loss = model.neg_log_likelihood(sentence_in, targets) # sent_in is list (noTokens); targets is torchArray (noTokens); # loss is single number
        #print("loss length"+str(len(loss))) 

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        print("before")
        print(model.lstm.weight_ih_l0)
        optimizer.step()
        print("after")
        print(model.lstm.weight_ih_l0)

    print("epoch:"+str(epoch))
    print(loss.item())

# Check predictions after training
#with torch.no_grad():
   #precheck_sent = prepare_sequence(training_data[0][0], word_to_ix)
    # print(model(precheck_sent))

before
Parameter containing:
tensor([[ 0.0857,  0.1884, -0.0090,  ..., -0.0296,  0.1132, -0.0199],
        [-0.1876, -0.0923, -0.1378,  ..., -0.1463,  0.0692,  0.1240],
        [ 0.0890,  0.1798,  0.1363,  ...,  0.0825,  0.0135,  0.1126],
        ...,
        [-0.1776,  0.0949,  0.0285,  ...,  0.1413, -0.1464,  0.0192],
        [ 0.1496,  0.1009,  0.0074,  ...,  0.1327,  0.1282, -0.0737],
        [-0.1389,  0.0154, -0.0694,  ..., -0.0498, -0.0443,  0.0603]],
       requires_grad=True)
after
Parameter containing:
tensor([[ 0.0858,  0.1883, -0.0090,  ..., -0.0298,  0.1132, -0.0202],
        [-0.1876, -0.0923, -0.1378,  ..., -0.1462,  0.0692,  0.1240],
        [ 0.0889,  0.1798,  0.1363,  ...,  0.0825,  0.0136,  0.1127],
        ...,
        [-0.1777,  0.0949,  0.0286,  ...,  0.1411, -0.1463,  0.0190],
        [ 0.1496,  0.1009,  0.0074,  ...,  0.1326,  0.1282, -0.0737],
        [-0.1388,  0.0154, -0.0694,  ..., -0.0499, -0.0443,  0.0604]],
       requires_grad=True)
before
Parameter cont

before
Parameter containing:
tensor([[ 0.0875,  0.1846, -0.0099,  ..., -0.0318,  0.1102, -0.0225],
        [-0.1910, -0.0963, -0.1390,  ..., -0.1487,  0.0731,  0.1195],
        [ 0.0893,  0.1796,  0.1369,  ...,  0.0832,  0.0142,  0.1128],
        ...,
        [-0.1765,  0.1000,  0.0272,  ...,  0.1414, -0.1431,  0.0202],
        [ 0.1487,  0.1001,  0.0068,  ...,  0.1334,  0.1279, -0.0747],
        [-0.1372,  0.0156, -0.0691,  ..., -0.0494, -0.0466,  0.0600]],
       requires_grad=True)
after
Parameter containing:
tensor([[ 0.0881,  0.1841, -0.0088,  ..., -0.0324,  0.1096, -0.0236],
        [-0.1921, -0.0974, -0.1398,  ..., -0.1489,  0.0771,  0.1176],
        [ 0.0894,  0.1794,  0.1367,  ...,  0.0832,  0.0142,  0.1129],
        ...,
        [-0.1762,  0.0999,  0.0270,  ...,  0.1413, -0.1417,  0.0205],
        [ 0.1484,  0.1001,  0.0065,  ...,  0.1329,  0.1280, -0.0749],
        [-0.1375,  0.0162, -0.0687,  ..., -0.0496, -0.0468,  0.0606]],
       requires_grad=True)
before
Parameter cont

before
Parameter containing:
tensor([[ 0.0888,  0.1870, -0.0139,  ..., -0.0283,  0.1082, -0.0263],
        [-0.1894, -0.0977, -0.1382,  ..., -0.1487,  0.0804,  0.1158],
        [ 0.0890,  0.1795,  0.1364,  ...,  0.0837,  0.0150,  0.1136],
        ...,
        [-0.1754,  0.0998,  0.0275,  ...,  0.1438, -0.1425,  0.0174],
        [ 0.1490,  0.1016,  0.0056,  ...,  0.1331,  0.1266, -0.0769],
        [-0.1370,  0.0164, -0.0675,  ..., -0.0504, -0.0454,  0.0608]],
       requires_grad=True)
after
Parameter containing:
tensor([[ 0.0885,  0.1871, -0.0145,  ..., -0.0276,  0.1088, -0.0257],
        [-0.1901, -0.0970, -0.1389,  ..., -0.1474,  0.0811,  0.1184],
        [ 0.0891,  0.1798,  0.1367,  ...,  0.0840,  0.0150,  0.1135],
        ...,
        [-0.1753,  0.0999,  0.0278,  ...,  0.1433, -0.1428,  0.0168],
        [ 0.1486,  0.1015,  0.0052,  ...,  0.1331,  0.1268, -0.0763],
        [-0.1373,  0.0166, -0.0683,  ..., -0.0502, -0.0442,  0.0615]],
       requires_grad=True)
before
Parameter cont

Parameter containing:
tensor([[ 0.0890,  0.1894, -0.0192,  ..., -0.0261,  0.1077, -0.0267],
        [-0.1920, -0.0941, -0.1384,  ..., -0.1496,  0.0786,  0.1187],
        [ 0.0891,  0.1796,  0.1360,  ...,  0.0842,  0.0147,  0.1140],
        ...,
        [-0.1745,  0.1024,  0.0273,  ...,  0.1443, -0.1425,  0.0169],
        [ 0.1492,  0.1021,  0.0054,  ...,  0.1347,  0.1261, -0.0767],
        [-0.1385,  0.0180, -0.0682,  ..., -0.0506, -0.0449,  0.0628]],
       requires_grad=True)
after
Parameter containing:
tensor([[ 0.0894,  0.1883, -0.0195,  ..., -0.0259,  0.1075, -0.0264],
        [-0.1924, -0.0931, -0.1393,  ..., -0.1501,  0.0789,  0.1187],
        [ 0.0894,  0.1797,  0.1360,  ...,  0.0844,  0.0143,  0.1142],
        ...,
        [-0.1739,  0.1024,  0.0273,  ...,  0.1441, -0.1424,  0.0176],
        [ 0.1492,  0.1026,  0.0055,  ...,  0.1347,  0.1262, -0.0766],
        [-0.1388,  0.0183, -0.0686,  ..., -0.0507, -0.0451,  0.0626]],
       requires_grad=True)
before
Parameter containing:

before
Parameter containing:
tensor([[ 0.0910,  0.1912, -0.0273,  ..., -0.0203,  0.1048, -0.0300],
        [-0.1966, -0.0893, -0.1379,  ..., -0.1460,  0.0781,  0.1213],
        [ 0.0897,  0.1817,  0.1352,  ...,  0.0850,  0.0148,  0.1144],
        ...,
        [-0.1751,  0.1055,  0.0269,  ...,  0.1455, -0.1404,  0.0191],
        [ 0.1493,  0.1027,  0.0046,  ...,  0.1364,  0.1264, -0.0789],
        [-0.1407,  0.0194, -0.0676,  ..., -0.0506, -0.0455,  0.0643]],
       requires_grad=True)
after
Parameter containing:
tensor([[ 0.0899,  0.1923, -0.0299,  ..., -0.0183,  0.1042, -0.0300],
        [-0.1961, -0.0898, -0.1382,  ..., -0.1466,  0.0775,  0.1226],
        [ 0.0896,  0.1819,  0.1352,  ...,  0.0853,  0.0156,  0.1147],
        ...,
        [-0.1738,  0.1058,  0.0271,  ...,  0.1446, -0.1407,  0.0184],
        [ 0.1492,  0.1029,  0.0053,  ...,  0.1372,  0.1258, -0.0785],
        [-0.1403,  0.0189, -0.0674,  ..., -0.0506, -0.0457,  0.0653]],
       requires_grad=True)
before
Parameter cont

...

before
Parameter containing:
tensor([[ 0.0884,  0.2012, -0.0476,  ..., -0.0099,  0.0989, -0.0340],
        [-0.2002, -0.0860, -0.1452,  ..., -0.1450,  0.0789,  0.1334],
        [ 0.0895,  0.1860,  0.1342,  ...,  0.0864,  0.0155,  0.1136],
        ...,
        [-0.1752,  0.1153,  0.0243,  ...,  0.1468, -0.1364,  0.0174],
        [ 0.1507,  0.1021,  0.0035,  ...,  0.1386,  0.1214, -0.0833],
        [-0.1450,  0.0242, -0.0696,  ..., -0.0509, -0.0467,  0.0656]],
       requires_grad=True)
after
Parameter containing:
tensor([[ 0.0883,  0.2011, -0.0470,  ..., -0.0100,  0.0973, -0.0344],
        [-0.2001, -0.0863, -0.1432,  ..., -0.1458,  0.0795,  0.1293],
        [ 0.0898,  0.1854,  0.1341,  ...,  0.0861,  0.0161,  0.1137],
        ...,
        [-0.1749,  0.1135,  0.0241,  ...,  0.1477, -0.1353,  0.0156],
        [ 0.1505,  0.1027,  0.0034,  ...,  0.1376,  0.1211, -0.0840],
        [-0.1447,  0.0239, -0.0689,  ..., -0.0491, -0.0477,  0.0681]],
       requires_grad=True)
before
Parameter cont

before
Parameter containing:
tensor([[ 0.0881,  0.2033, -0.0503,  ..., -0.0091,  0.1002, -0.0351],
        [-0.2003, -0.0824, -0.1447,  ..., -0.1465,  0.0793,  0.1332],
        [ 0.0896,  0.1859,  0.1343,  ...,  0.0855,  0.0156,  0.1140],
        ...,
        [-0.1754,  0.1126,  0.0250,  ...,  0.1503, -0.1366,  0.0149],
        [ 0.1490,  0.1025,  0.0037,  ...,  0.1372,  0.1206, -0.0835],
        [-0.1469,  0.0232, -0.0708,  ..., -0.0498, -0.0488,  0.0663]],
       requires_grad=True)
after
Parameter containing:
tensor([[ 0.0880,  0.2044, -0.0517,  ..., -0.0080,  0.1001, -0.0361],
        [-0.1994, -0.0807, -0.1448,  ..., -0.1466,  0.0788,  0.1330],
        [ 0.0900,  0.1864,  0.1342,  ...,  0.0852,  0.0159,  0.1142],
        ...,
        [-0.1752,  0.1121,  0.0251,  ...,  0.1499, -0.1346,  0.0152],
        [ 0.1478,  0.1017,  0.0040,  ...,  0.1381,  0.1204, -0.0844],
        [-0.1461,  0.0238, -0.0707,  ..., -0.0493, -0.0486,  0.0666]],
       requires_grad=True)
before
Parameter cont

before
Parameter containing:
tensor([[ 8.6247e-02,  2.0586e-01, -5.7614e-02,  ..., -7.9797e-05,
          1.0051e-01, -3.9495e-02],
        [-1.9723e-01, -7.5784e-02, -1.4721e-01,  ..., -1.4826e-01,
          8.2445e-02,  1.3981e-01],
        [ 9.0859e-02,  1.8646e-01,  1.3378e-01,  ...,  8.6241e-02,
          1.5746e-02,  1.1384e-01],
        ...,
        [-1.7604e-01,  1.1401e-01,  2.5431e-02,  ...,  1.4796e-01,
         -1.3238e-01,  1.7244e-02],
        [ 1.4717e-01,  1.0021e-01,  3.2419e-03,  ...,  1.3788e-01,
          1.2080e-01, -8.5312e-02],
        [-1.4588e-01,  2.5333e-02, -7.1618e-02,  ..., -4.9129e-02,
         -4.6999e-02,  6.7798e-02]], requires_grad=True)
after
Parameter containing:
tensor([[ 0.0851,  0.2064, -0.0583,  ...,  0.0006,  0.1001, -0.0393],
        [-0.1978, -0.0752, -0.1476,  ..., -0.1485,  0.0826,  0.1394],
        [ 0.0911,  0.1866,  0.1338,  ...,  0.0865,  0.0152,  0.1140],
        ...,
        [-0.1754,  0.1136,  0.0254,  ...,  0.1481, -0.1325,  0.0169]

before
Parameter containing:
tensor([[ 8.3149e-02,  2.0698e-01, -6.1292e-02,  ...,  4.1587e-03,
          1.0129e-01, -3.9616e-02],
        [-1.9797e-01, -7.2117e-02, -1.5054e-01,  ..., -1.5197e-01,
          8.3378e-02,  1.4148e-01],
        [ 9.0902e-02,  1.8863e-01,  1.3386e-01,  ...,  8.6330e-02,
          1.5120e-02,  1.1419e-01],
        ...,
        [-1.7499e-01,  1.1273e-01,  2.5357e-02,  ...,  1.5093e-01,
         -1.2972e-01,  1.5952e-02],
        [ 1.4864e-01,  9.9604e-02,  7.0799e-05,  ...,  1.3892e-01,
          1.2086e-01, -8.7210e-02],
        [-1.4431e-01,  2.8561e-02, -7.2458e-02,  ..., -4.9653e-02,
         -5.0374e-02,  6.7313e-02]], requires_grad=True)
after
Parameter containing:
tensor([[ 0.0839,  0.2081, -0.0627,  ...,  0.0048,  0.1019, -0.0407],
        [-0.1991, -0.0722, -0.1511,  ..., -0.1529,  0.0836,  0.1420],
        [ 0.0908,  0.1889,  0.1342,  ...,  0.0865,  0.0153,  0.1144],
        ...,
        [-0.1761,  0.1125,  0.0246,  ...,  0.1519, -0.1265,  0.0165]

before
Parameter containing:
tensor([[ 0.0814,  0.2113, -0.0636,  ...,  0.0073,  0.1020, -0.0451],
        [-0.2017, -0.0746, -0.1537,  ..., -0.1520,  0.0892,  0.1452],
        [ 0.0915,  0.1900,  0.1331,  ...,  0.0869,  0.0131,  0.1139],
        ...,
        [-0.1731,  0.1125,  0.0213,  ...,  0.1551, -0.1243,  0.0159],
        [ 0.1492,  0.0979, -0.0009,  ...,  0.1398,  0.1187, -0.0888],
        [-0.1456,  0.0297, -0.0751,  ..., -0.0497, -0.0489,  0.0649]],
       requires_grad=True)
after
Parameter containing:
tensor([[ 0.0805,  0.2123, -0.0642,  ...,  0.0084,  0.1028, -0.0464],
        [-0.2008, -0.0743, -0.1526,  ..., -0.1532,  0.0888,  0.1452],
        [ 0.0915,  0.1902,  0.1334,  ...,  0.0866,  0.0122,  0.1141],
        ...,
        [-0.1736,  0.1128,  0.0228,  ...,  0.1551, -0.1229,  0.0157],
        [ 0.1491,  0.0979, -0.0011,  ...,  0.1404,  0.1182, -0.0884],
        [-0.1456,  0.0296, -0.0746,  ..., -0.0503, -0.0492,  0.0644]],
       requires_grad=True)
before
Parameter cont

In [13]:
# generate predictions v2
sentences = [a for a,_ in test_data]
labels = [b for _,b in test_data]
#print(prepare_sequence(sentences[0], word_to_ix))
#test_data_preds = model(prepare_sequence(sentences[0], word_to_ix))
#print(test_data_preds)
#print(labels[0])
with torch.no_grad():
    test_data_preds = [model(prepare_sequence(a, word_to_ix)) for a in sentences]
    test_data_preds = [b for _,b in test_data_preds]
    test_data_preds_l = [[torch.IntTensor.item(x) for x in item] for item in test_data_preds] # list of list of ints
    test_data_gold = [[tag_to_ix[el] for el in a] for a in labels] 

In [15]:
ix_to_tag = {value: key for key, value in tag_to_ix.items()}

In [16]:
labels_distinct = sorted(list(tag_to_ix.keys()))

In [17]:
# don't unfold but convert ids to tags:
test_data_preds_tag = [[str(ix_to_tag[int(item)]) for item in sublist] for sublist in test_data_preds_l]
test_data_gold_tag = [[str(ix_to_tag[int(item)]) for item in sublist] for sublist in test_data_gold ]

In [18]:
print(metrics.flat_classification_report(
    test_data_gold_tag, test_data_preds_tag, labels=labels_distinct, digits=3
))

                     precision    recall  f1-score   support

            <START>      0.000     0.000     0.000         0
             <STOP>      0.000     0.000     0.000         0
DATUM_VERBUECHERUNG      0.667     0.873     0.756        55
      DATUM_VERTRAG      0.808     0.677     0.737        62
            FLAECHE      0.364     0.105     0.163        38
        GESAMTPREIS      0.138     0.138     0.138        29
           IMMO_TYP      0.167     0.085     0.113        47
            KAEUFER      0.333     0.182     0.235        33
                  O      0.882     0.971     0.924      1525
                ORT      0.717     0.644     0.679        59
            QMPREIS      0.000     0.000     0.000        21
            STRASSE      0.552     0.364     0.438        44
   TERRASSENGROESSE      0.000     0.000     0.000         8
         VERKAEUFER      0.200     0.048     0.078        62

          micro avg      0.830     0.830     0.830      1983
          macro avg   

C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\utils\validation.py:67: FutureWarning: Pass labels=['<START>', '<STOP>', 'DATUM_VERBUECHERUNG', 'DATUM_VERTRAG', 'FLAECHE', 'GESAMTPREIS', 'IMMO_TYP', 'KAEUFER', 'O', 'ORT', 'QMPREIS', 'STRASSE', 'TERRASSENGROESSE', 'VERKAEUFER'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "
C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\vasco\anaconda3\envs\xcs224u\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_